Battery life and thermal dissipation simulation.
(C) 2024, CC-BY-NC-SA 4.0

In [1]:
## LICENCE: (C) 2024, CC-BY-NC-SA 4.0

#               rail    Vout    Vin    efficiency
supply_table = {"3V3" : (3.3,   3.7,   0.9),
                "1V8" : (1.8,   3.7,   0.9),
                "1V2" : (1.2,   3.7,   0.9)}
#               load                    rail    Power state : draw (mA)
load_table   =  {"DWM1000_3V3"      : ("3V3", {"RX": 160, "TX" : 140, "IDLE" : 13.4, "SLEEP" : 0.00055, "DEEP SLEEP" : 0.0001}),
                 "ZED-F9P-15B_3V3"  : ("3V3", { "Aquisition" : 90, "Tracking" : 85, "Backup" : 1.4}),
                 "317990687_3V3"  : ("3V3", {"Sleep" : 0.0021, "Recv" : 6.7, "Trans" : 111}),
                 "NRF9160_3V3"  : ("3V3", {"SLEEP" : 0.6, "MODEM" : 115, "GPS" : 44.3,  "CPU only" : 2.2}),
                 "SD_3V3"  : ("3V3", {"WRITE" : 40, "SLEEP" : 0.5}),
                 "TMS5701224CPGEQQ1_1V2"  : ("1V2", {"180M": 190, "160M" : 170, "80M" : 170-80, "40M" : 170-120, "Doze" : 17.3})

}

AMBIENT_TEMP = 60 #°C

#               part        thetaJA (°C/W) list supplies
thermal_table = [("NRF9160",           35, ["NRF9160_3V3"]),
                 ("TMS5701224CPGEQQ1", 40, ["TMS5701224CPGEQQ1_1V2"]),
                 ("LT3154 3.3 V", 30, ["3V3"]),
                 ("LT3154 1.2 V", 30, ["1V2"]),
                 ("LT3154 1.8 V", 30, ["1V8"])]

LTE_DATARATE = 100000 #bit/s
LORA_DATARATE = 100000 #bit/s
UWB_DATARATE = 110000 #bit/s
LTE_CONTINOUS = 5000/LTE_DATARATE
LORA_CONTINOUS = 5000/LORA_DATARATE
LTE_PAYLOAD = 1000 #bytes
LORA_PAYLOAD = 1000 #bytes
UWB_PAYLOAD = 1000 #bytes
LTE_INTERVAL = 60 #seconds
LORA_INTERVAL = 60 #seconds
UWB_INTERVAL = 60 #seconds
LTE_BURST = (8*LTE_PAYLOAD/LTE_DATARATE)/LTE_INTERVAL
LORA_BURST = (8*LORA_PAYLOAD/LORA_DATARATE)/LORA_INTERVAL
UWB_BURST= (8*UWB_PAYLOAD/UWB_DATARATE)/UWB_INTERVAL

GNSS_FIX_TIME = 5 #seconds
GNSS_INTERVAL = 60 #seconds
GNSS_BURST = GNSS_FIX_TIME/GNSS_INTERVAL

SD_DATARATE = 20000000 #bit/s
SD_PAYLOAD = 1000 #bytes
SD_INTERVAL = 60 #seconds
SD_BURST = (8*SD_PAYLOAD/SD_DATARATE)/SD_INTERVAL

profile_table = {"IN FLIGHT" :
                 # load            list (power state, duty cycle)
                 {"DWM1000_3V3" : [("SLEEP" , 1)],
                  "ZED-F9P-15B_3V3" : [("Tracking", 1)],
                  "TMS5701224CPGEQQ1_1V2" : [("80M", 1)],
                  "NRF9160_3V3" : [("MODEM", LTE_CONTINOUS), ("GPS", 1)],
                  "317990687_3V3"  : [("Recv", LORA_CONTINOUS), ("Trans", LORA_CONTINOUS), ("Sleep", 1)],
                  "SD_3V3" : [("WRITE", 1)]},
                 "BEACON" :
                 {"DWM1000_3V3" : [("RX" , UWB_BURST), ("TX", UWB_BURST), ("IDLE", 0), ("SLEEP", 1-2*UWB_BURST)],
                  "ZED-F9P-15B_3V3" : [("Aquisition", GNSS_BURST), ("Backup", 1-GNSS_BURST)],
                   "317990687_3V3"  : [("Recv", LORA_BURST), ("Trans", LORA_BURST), ("Sleep", 1-2*LORA_BURST)],
                  "NRF9160_3V3" : [("MODEM", LTE_BURST), ("GPS", GNSS_BURST), ("SLEEP", 1-GNSS_BURST-LTE_BURST)],
                  "TMS5701224CPGEQQ1_1V2" : [("80M", 0.1), ("Doze", 0.9)],
                  "SD_3V3" : [("WRITE", SD_BURST), ("SLEEP", 1-SD_BURST)]}

}

draw_table, dissipation_table = {}, {}

for profile in profile_table.keys():
  print("Estimating current draw for profile: ", profile, "\n")
  total_draw = {}
  supply_dissipation = {}
  battery_current = 0
  for load in profile_table[profile].keys():
    current_draw = sum([duty_cycle*load_table[load][1][state]
                        for state, duty_cycle in profile_table[profile][load]])
    power_draw = current_draw * supply_table[load_table[load][0]][0]
    dissipation_table[load] = power_draw
    print("Load: ", load, " draws: ", current_draw,
          " mA, dissipates", power_draw, " mW")
    total_draw[load_table[load][0]] = current_draw + total_draw.get(
      load_table[load][0], 0)
  for supply in total_draw.keys():
    Vout, Vin, eff = supply_table[supply]
    supply_dissipation[supply] = (Vout*total_draw[supply]/eff) -\
      Vout*total_draw[supply]
    battery_current += (Vout*total_draw[supply]/eff)/Vin
  print("\nTemperature rises for ambient temp, ", AMBIENT_TEMP, " °C")
  for part, theta, supplies in thermal_table:
    p = sum([dissipation_table.get(supply, 0) for supply in supplies]) +\
        sum([supply_dissipation.get(supply, 0) for supply in supplies])
    delta = 0.001*p*theta
    print(part, " : delta T = ", delta, " °C, T = ",
          delta + AMBIENT_TEMP, " °C")
  print("Total draws, ", total_draw, " mA")
  print("Supply dissipation, ", supply_dissipation, " mW")
  print("Total battery current, ", battery_current, " mA\n")
  draw_table[profile] = battery_current

print("For 24 hour operation, the required capacity is: ",
      draw_table["IN FLIGHT"]*0.25 + 23.75*draw_table["BEACON"], "mAh\n")


Estimating current draw for profile:  IN FLIGHT 

Load:  DWM1000_3V3  draws:  0.00055  mA, dissipates 0.001815  mW
Load:  ZED-F9P-15B_3V3  draws:  85  mA, dissipates 280.5  mW
Load:  TMS5701224CPGEQQ1_1V2  draws:  90  mA, dissipates 108.0  mW
Load:  NRF9160_3V3  draws:  50.05  mA, dissipates 165.165  mW
Load:  317990687_3V3  draws:  5.887100000000001  mA, dissipates 19.42743  mW
Load:  SD_3V3  draws:  40  mA, dissipates 132.0  mW

Temperature rises for ambient temp,  60  °C
NRF9160  : delta T =  5.780775  °C, T =  65.780775  °C
TMS5701224CPGEQQ1  : delta T =  4.32  °C, T =  64.32  °C
LT3154 3.3 V  : delta T =  1.9903141499999992  °C, T =  61.990314149999996  °C
LT3154 1.2 V  : delta T =  0.36  °C, T =  60.36  °C
LT3154 1.8 V  : delta T =  0.0  °C, T =  60.0  °C
Total draws,  {'3V3': 180.93765, '1V2': 90}  mA
Supply dissipation,  {'3V3': 66.34380499999997, '1V2': 12.0}  mW
Total battery current,  211.74001351351345  mA

Estimating current draw for profile:  BEACON 

Load:  DWM1000_3V3  